In [1]:
from datetime import datetime
import pytz

def fecha_hora_lima():
    lima_tz = pytz.timezone('America/Lima')
    lima_time = datetime.now(lima_tz)
    return lima_time.strftime('%Y%m%d-%H%M%S')

def fecha_hora_lima_db():
    lima_tz = pytz.timezone('America/Lima')
    lima_time = datetime.now(lima_tz)
    return lima_time.strftime('%Y-%m-%d %H:%M:%S')

fecha_hora_para_archivos = fecha_hora_lima()
fecha_hora_para_base_de_datos = fecha_hora_lima_db()

In [2]:
import pandas as pd
import requests
import json

# URL del JSON
url = 'https://www.wong.pe/api/catalog_system/pub/category/tree/3'

# Solicitar el contenido JSON desde la URL
response = requests.get(url)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    data = response.json()  # Convertir la respuesta en un objeto JSON
    file_name = f'categories_{fecha_hora_para_archivos}.json'
    
    # Guardar el JSON en un archivo
    with open(file_name, 'w') as file:
        json.dump(data, file, indent=4)

    print('El archivo JSON se ha guardado exitosamente.')
    with open(file_name, 'r') as file:
        data = json.load(file)
    print(len(data)) 
else:
    print(f'Error al obtener el JSON. Código de estado: {response.status_code}')  

El archivo JSON se ha guardado exitosamente.
26


In [3]:
# file_path = 'wong_categories.json'
# 
# with open(file_path, 'r') as file:
#     data = json.load(file)
#     print(json.dumps(data, indent=4))

In [4]:
columns = ['id', 'name', 'hasChildren', 'url', 'NumChildren', 'Title', 'MetaTagDescription', 'parent_id']
df = pd.DataFrame(columns=columns)
df

,id,name,hasChildren,url,NumChildren,Title,MetaTagDescription,parent_id


In [5]:
for item in data:
  if item['hasChildren']:
    item['parent_id'] = ''
    item['NumChildren'] = len(item['children'])

    df.loc[len(df)] = item

    parent_id = item['id']

    for subitem in item['children']:
      subitem['parent_id'] = parent_id
      sub_parent_id = subitem['id']
      subitem['NumChildren'] = len(subitem['children'])

      df.loc[len(df)] = subitem

      if len(subitem['children']) > 0:
        for subsubitem in subitem['children']:
          subsubitem['parent_id'] = sub_parent_id
          subsubitem['NumChildren'] = len(subsubitem['children'])
          df.loc[len(df)] = subsubitem

In [6]:
df.query("id == 952")

,id,name,hasChildren,url,NumChildren,Title,MetaTagDescription,parent_id


In [7]:
file_name_categories_csv = f'wong_categories_{fecha_hora_para_archivos}.csv'
df.to_csv(file_name_categories_csv, index=False)

In [8]:
cats = df.query("parent_id == ''")['id'].to_list()
print(cats)

[800, 1200, 1400, 1700, 1800, 1900, 2000, 2100, 2200, 1000144, 1000003, 1000131, 1000143, 1000069, 1001074, 1001253, 1001327, 2301, 1001374, 1001402, 1001436, 1001467, 1001470, 1001673, 1001701]


In [9]:
import time

products = []

for cat in cats:
    start = 0
    end = 49
    
    while True:
        time.sleep(1)
        url = f"https://www.wong.pe/api/catalog_system/pub/products/search?fq=C:{cat}&_from={start}&_to={end}&O=OrderByPriceDESC"
        
        response = requests.get(url)
        
        if response.status_code == 200 or response.status_code == 206:
            data = response.json()
            if not data:
                break
                 
            products.extend(data)
            start += 50
            end += 50
        else:
            break
    
    print(f"status_code = {response.status_code}")
    print(f"cat = {cat}, start = {start}, end = {end}")

 
file_name_productos_csv = f'productos_{fecha_hora_para_archivos}.json'

with open(file_name_productos_csv, 'w') as file:
    json.dump(products, file, indent=4)

status_code = 200
cat = 800, start = 600, end = 649
status_code = 200
cat = 1200, start = 600, end = 649
status_code = 200
cat = 1400, start = 550, end = 599
status_code = 500
cat = 1700, start = 2250, end = 2299
status_code = 200
cat = 1800, start = 650, end = 699
status_code = 200
cat = 1900, start = 1200, end = 1249
status_code = 200
cat = 2000, start = 1000, end = 1049
status_code = 400
cat = 2100, start = 2550, end = 2599
status_code = 400
cat = 2200, start = 2550, end = 2599
status_code = 200
cat = 1000144, start = 300, end = 349
status_code = 200
cat = 1000003, start = 350, end = 399
status_code = 400
cat = 1000131, start = 2550, end = 2599
status_code = 200
cat = 1000143, start = 1250, end = 1299
status_code = 200
cat = 1000069, start = 600, end = 649
status_code = 200
cat = 1001074, start = 2250, end = 2299
status_code = 200
cat = 1001253, start = 1450, end = 1499
status_code = 200
cat = 1001327, start = 750, end = 799
status_code = 200
cat = 2301, start = 2150, end = 2199
sta

In [10]:
print(len(products))

27336


In [11]:
columns = ['productId', 'productName', 'brand', 'brandId', 'brandImageUrl','categoryId',
           'productTitle', 'metaTagDescription', 'categories', 'categoriesIds', 'link', 'Vendido por',
           'Descripción del producto', 'Presentación', 'Presentación/Empaque', 'Pack', 'Porción Sugerida',
           'Azúcares Por Porción', 'Carbohidratos Por Porción', 'Descripción Nutricional', 'Grasas Por Porción',
           'Advertencias de consumo', 'itemId', 'ean', 'imageUrl', 'Value', 'IsAvailable', 'AvailableQuantity',
           'Price', 'ListPrice', 'PriceWithoutDiscount', 'PriceValidUntil', 'Created']

df = pd.DataFrame(columns=columns)
df

,productId,productName,brand,brandId,brandImageUrl,categoryId,productTitle,metaTagDescription,categories,categoriesIds,...,ean,imageUrl,Value,IsAvailable,AvailableQuantity,Price,ListPrice,PriceWithoutDiscount,PriceValidUntil,Created


In [12]:
for product in products:
  print(product['items'][0]['sellers'][0]['commertialOffer']['Installments'][0]['Value'])
  print(product['items'][0]['sellers'][0]['commertialOffer']['PriceValidUntil'])
  print(product['items'][0]['sellers'][0]['commertialOffer']['ListPrice'])
  print(product['items'][0]['sellers'][0]['commertialOffer']['PriceWithoutDiscount'])
  break

279.0
2100-01-01T04:59:59Z
279.0
279.0


In [13]:
print(json.dumps(products[3099], indent=4))

{
    "productId": "554577",
    "productName": "Fudge Heladero Leite Frasco 400 g",
    "brand": "Leite",
    "brandId": 1562,
    "brandImageUrl": null,
    "linkText": "fudge-heladero-leite-frasco-400-g-2-2",
    "productReference": "789800",
    "productReferenceCode": "789800",
    "categoryId": "1000209",
    "productTitle": null,
    "metaTagDescription": null,
    "releaseDate": "2024-09-27T00:00:00Z",
    "clusterHighlights": {
        "7035": "TC Abarrotes y Perecibles",
        "7083": "Agenda Ahorro Wong Lunes",
        "7086": "Agenda Ahorro Metro Lunes",
        "8484": "Semana Repostera Wong Complementos",
        "10741": "Dinamica Paneton Metro",
        "10899": "Delivery Gratis T121 Playas",
        "10901": "Delivery Gratis T116 Playas",
        "11044": "Abarrotes Wong",
        "11069": "Supermercado Metro",
        "11746": "Abarrotes wong audiencia promo paneton",
        "11747": "Abarrotes metro Audiencia promo paneton",
        "11748": "Supermercados Wong Fo

In [14]:
# print(json.dumps(products, indent=4))

In [15]:
def print_value(value, key):
    if value:
        print(f"{key} = {value}")
    else:
        print(f"{key} = None")

In [16]:
from bs4 import BeautifulSoup

def clean_html(html_code):
    if html_code is None:
        return ''
    if isinstance(html_code, list):
        html_code = ' '.join(html_code)
    soup = BeautifulSoup(html_code, 'html.parser')
    return soup.get_text()

In [17]:
def get_product_data(product, fecha_db):
    product['Created'] = fecha_db
    new_product = {}
    new_product['productId'] = product.get('productId')
    new_product['productName'] = product.get('productName')
    new_product['brand'] = product.get('brand')
    new_product['brandId'] = product.get('brandId')
    new_product['brandImageUrl'] = product.get('brandImageUrl')
    new_product['categoryId'] = product.get('categoryId')
    new_product['productTitle'] = product.get('productTitle')
    new_product['metaTagDescription'] = product.get('metaTagDescription')
    new_product['categories'] = product.get('categories')
    new_product['categoriesIds'] = product.get('categoriesIds')
    new_product['link'] = product.get('link')
    new_product['Vendido por'] = product.get('Vendido por')
    new_product['Descripción del producto'] = clean_html(product.get('Descripción del producto'))
    new_product['Presentación'] = product.get('Presentación')
    new_product['Presentación/Empaque'] = product.get('Presentación/Empaque')
    new_product['Pack'] = product.get('Pack')
    new_product['Porción Sugerida'] = product.get('Porción Sugerida')
    new_product['Azúcares Por Porción'] = product.get('Azúcares Por Porción')
    new_product['Carbohidratos Por Porción'] = product.get('Carbohidratos Por Porción')
    new_product['Descripción Nutricional'] = product.get('Descripción Nutricional')
    new_product['Grasas Por Porción'] = product.get('Grasas Por Porción')
    new_product['Advertencias de consumo'] = product.get('Advertencias de consumo')
    
    if product['items']:
        new_product['itemId'] = product['items'][0].get('itemId', None)
        new_product['ean'] = product['items'][0].get('ean', None)
        if product['items'][0]['images']:
            new_product['imageUrl'] = product['items'][0]['images'][0].get('imageUrl', None)
            if product['items'][0]['sellers']:
                commertial_offer = product['items'][0]['sellers'][0].get('commertialOffer', {})
                if commertial_offer.get('Installments'):
                    new_product['Value'] = commertial_offer['Installments'][0].get('Value', None)
                new_product['IsAvailable'] = commertial_offer.get('IsAvailable', None)
                new_product['AvailableQuantity'] = commertial_offer.get('AvailableQuantity', None)
                new_product['Price'] = commertial_offer.get('Price', None)
                new_product['ListPrice'] = commertial_offer.get('ListPrice', None)
                new_product['PriceWithoutDiscount'] = commertial_offer.get('PriceWithoutDiscount', None)
                new_product['PriceValidUntil'] = commertial_offer.get('PriceValidUntil', None)

    new_product['Created'] = product['Created']
    
    return new_product

In [18]:
# Para validar que los datos se están extrayendo correctamente

i = 0
for product in products:
    i += 1
    product['Created'] = fecha_hora_lima_db()
    print_value(product.get('productId'), 'productId')
    print_value(product.get('productName'), 'productName')
    print_value(product.get('brand'), 'brand')
    print_value(product.get('brandId'), 'brandId')
    print_value(product.get('brandImageUrl'), 'brandImageUrl')
    print_value(product.get('categoryId'), 'categoryId')
    print_value(product.get('productTitle'), 'productTitle')
    print_value(product.get('metaTagDescription'), 'metaTagDescription')
    print_value(product.get('categories'), 'categories')
    print_value(product.get('categoriesIds'), 'categoriesIds')
    print_value(product.get('link'), 'link')
    print_value(product.get('Vendido por'), 'Vendido por')
    print_value(clean_html(product.get('Descripción del producto')), 'Descripción del producto')
    print_value(product.get('Presentación'), 'Presentación')
    print_value(product.get('Presentación/Empaque'), 'Presentación/Empaque')
    print_value(product.get('Pack'), 'Pack')
    print_value(product.get('Porción Sugerida'), 'Porción Sugerida')
    print_value(product.get('Azúcares Por Porción'), 'Azúcares Por Porción')
    print_value(product.get('Carbohidratos Por Porción'), 'Carbohidratos Por Porción')
    print_value(product.get('Descripción Nutricional'), 'Descripción Nutricional')
    print_value(product.get('Grasas Por Porción'), 'Grasas Por Porción')
    print_value(product.get('Advertencias de consumo'), 'Advertencias de consumo')
    print_value(product['items'][0]['itemId'], 'itemId')
    print_value(product['items'][0]['ean'], 'ean')
    print_value(product['items'][0]['images'][0]['imageUrl'], 'imageUrl')
    print_value(product['items'][0]['sellers'][0]['commertialOffer']['Installments'][0]['Value'], 'Value')
    print_value(product['items'][0]['sellers'][0]['commertialOffer']['IsAvailable'], 'IsAvailable')
    print_value(product['items'][0]['sellers'][0]['commertialOffer']['AvailableQuantity'], 'AvailableQuantity')
    print_value(product['items'][0]['sellers'][0]['commertialOffer']['Price'], 'Price')
    print_value(product['items'][0]['sellers'][0]['commertialOffer']['ListPrice'], 'ListPrice')
    print_value(product['items'][0]['sellers'][0]['commertialOffer']['PriceWithoutDiscount'], 'PriceWithoutDiscount')
    print_value(product['items'][0]['sellers'][0]['commertialOffer']['PriceValidUntil'], 'PriceValidUntil')
    print_value(product['Created'], 'Created')
    
    if i == 3:
        break

productId = 939222
productName = Fruta Deshidratada Manzana, Fresa y Melón Natifrut x kg
brand = Genérico
brandId = 9619
brandImageUrl = None
categoryId = 1001317
productTitle = Fruta Deshidratada Manzana, Fresa y Melón Natifrut x kg
metaTagDescription = ¡Encuentra aquí la mejor variedad de Frutos Secos a Granel de la marca Natifrut!
categories = ['/Frutas y Verduras/Frutas/Frutos Secos/', '/Frutas y Verduras/Frutas/', '/Frutas y Verduras/']
categoriesIds = ['/800/801/1001317/', '/800/801/', '/800/']
link = https://www.wong.pe/fruta-deshidratada-manzana-fresa-y-melon-natifrut-x-kg-771048-2/p
Vendido por = None
Descripción del producto = None
Presentación = None
Presentación/Empaque = None
Pack = None
Porción Sugerida = None
Azúcares Por Porción = None
Carbohidratos Por Porción = None
Descripción Nutricional = None
Grasas Por Porción = None
Advertencias de consumo = None
itemId = 39179921
ean = 22415
imageUrl = https://wongfood.vteximg.com.br/arquivos/ids/509042/Mix-Manzana-Fresa-Mel-n-

In [20]:
print(len(products))

27336


In [21]:
i=0
for product in products:
    i+=1
    if i % 100 == 0:
        print("Iteración #: ",i)
    new_product = get_product_data(product, fecha_hora_para_base_de_datos)
    df.loc[len(df)] = new_product

Iteración #:  100
Iteración #:  200
Iteración #:  300
Iteración #:  400
Iteración #:  500
Iteración #:  600
Iteración #:  700
Iteración #:  800
Iteración #:  900
Iteración #:  1000
Iteración #:  1100
Iteración #:  1200
Iteración #:  1300
Iteración #:  1400
Iteración #:  1500
Iteración #:  1600
Iteración #:  1700
Iteración #:  1800
Iteración #:  1900
Iteración #:  2000
Iteración #:  2100
Iteración #:  2200
Iteración #:  2300
Iteración #:  2400
Iteración #:  2500
Iteración #:  2600
Iteración #:  2700
Iteración #:  2800
Iteración #:  2900
Iteración #:  3000
Iteración #:  3100
Iteración #:  3200
Iteración #:  3300
Iteración #:  3400
Iteración #:  3500
Iteración #:  3600
Iteración #:  3700
Iteración #:  3800
Iteración #:  3900
Iteración #:  4000
Iteración #:  4100
Iteración #:  4200
Iteración #:  4300
Iteración #:  4400
Iteración #:  4500
Iteración #:  4600
Iteración #:  4700
Iteración #:  4800
Iteración #:  4900
Iteración #:  5000
Iteración #:  5100
Iteración #:  5200
Iteración #:  5300
It

In [22]:
df.sample(n=5)

,productId,productName,brand,brandId,brandImageUrl,categoryId,productTitle,metaTagDescription,categories,categoriesIds,...,ean,imageUrl,Value,IsAvailable,AvailableQuantity,Price,ListPrice,PriceWithoutDiscount,PriceValidUntil,Created
35281,969033,Yogurt Griego Vakimu Mora 500g,Vakimu,2001498,None,1316,Yogurt Griego Vakimu Mora 500g,¡Encuentra aquí la mejor variedad de Yogurt Gr...,[/Lácteos/Yogures/Yogurt Probiótico y Funcion...,"[/1001436/1001442/1316/, /1001436/1001442/, /1...",...,7756305001329,https://wongfood.vteximg.com.br/arquivos/ids/5...,8.41,True,100,8.41,9.9,8.41,2024-10-21T14:00:00Z,2024-10-14 19:22:40
3495,1054909,Twopack Gelatina en Polvo Royal Sabor Limón 160g,Royal,786,None,1001306,Twopack Gelatina en Polvo Royal Sabor Limón 160g,¡Encuentra aquí los mejores productos de Gelat...,[/Abarrotes/Repostería/Gelatinas y Mazamorras/...,"[/1700/1000694/1001306/, /1700/1000694/, /1700/]",...,2050044343111,https://wongfood.vteximg.com.br/arquivos/ids/7...,10.40,True,99999,10.40,10.4,10.40,2100-01-01T04:59:59Z,2024-10-14 19:22:40
18975,36581,Vino Tinto Blend Sottano Botella 750ml,Sottano,8641,None,2113,Vino Tinto Blend Sottano Botella 750ml,None,"[/Cervezas, Vinos y Licores/Vinos/Vino Tinto/,...","[/2100/2101/2113/, /2100/2101/, /2100/]",...,7798127631322,https://wongfood.vteximg.com.br/arquivos/ids/5...,52.90,True,100,52.90,75.9,52.90,2100-01-01T04:59:59Z,2024-10-14 19:22:40
19955,1040657,Twopack Zinc 50mg 90 Tabletas,Generico,1043,None,1001235,Twopack Zinc 50mg 90 Tabletas,¡Encuentra aquí los mejores productos de Algod...,"[/Higiene, Salud y Belleza/Salud/Algodón y Bot...","[/2200/1000782/1001235/, /2200/1000782/, /2200/]",...,840093103193,https://wongfood.vteximg.com.br/arquivos/ids/6...,92.90,True,100,92.90,92.9,92.90,2100-01-01T04:59:59Z,2024-10-14 19:22:40
2527,1011140,Galletas de Avena Sabor a Naranja Cuisine & Co...,Cuisine & Co,9885,None,1001429,Galletas de Avena Sabor a Naranja Cuisine & Co...,¡Encuentra aquí los mejores productos de Galle...,"[/Abarrotes/Galletas, Snacks y Golosinas/Galle...","[/1700/1001424/1001429/, /1700/1001424/, /1700/]",...,2050044184653,https://wongfood.vteximg.com.br/arquivos/ids/6...,18.90,True,100,18.90,18.9,18.90,2100-01-01T04:59:59Z,2024-10-14 19:22:40


In [23]:
df[['productId', 'productName', 'brand', 'categoryId', 'link', 'Price', 'AvailableQuantity']].sample(n=8)

,productId,productName,brand,categoryId,link,Price,AvailableQuantity
27217,1038778,Dormitorio Royal Abrazzo Cajones 2 Plazas Cabe...,Paraiso,1001094,https://www.wong.pe/dormitorio-royal-abrazzo-c...,3299.00,100
33077,30420,Resaltador Ice 1546 Amarillo,Faber Castell,1000879,https://www.wong.pe/resaltador-ice-1546-amaril...,5.19,100
5504,552053,Paños Multiusos Home Care 3un,Home Care,1920,https://www.wong.pe/pa-os-multiusos-home-care-...,7.90,100
31355,1032891,Mixtura de Mariscos Royal Seafrost 300g,Seafrost,1001362,https://www.wong.pe/mixtura-de-mariscos-royal-...,19.90,100
3985,953130,Jugo de Toronja Florida's 1.5L,Floridas Natural,1001162,https://www.wong.pe/jugo-de-toronja-florida-s-...,19.90,100
7240,50721,Ron Carúpano Reserva Exclusiva 12 Años Botella...,Carupano,2104,https://www.wong.pe/ron-carupano-reserva-exclu...,113.90,100
37419,1043790,Peluche Ty Beanie Boos Eva Elefante Reg,TY,1001169,https://www.wong.pe/peluche-ty-beanie-boos-eva...,24.90,100
19121,4780,Vino Blanco Blend Ruffino Orvieto Classico DOC...,Ruffino,2113,https://www.wong.pe/vino-blanco-blend-ruffino-...,48.30,100


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37840 entries, 0 to 37839
Data columns (total 33 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   productId                  37840 non-null  object 
 1   productName                37840 non-null  object 
 2   brand                      37840 non-null  object 
 3   brandId                    37840 non-null  int64  
 4   brandImageUrl              0 non-null      object 
 5   categoryId                 37840 non-null  object 
 6   productTitle               30220 non-null  object 
 7   metaTagDescription         24313 non-null  object 
 8   categories                 37840 non-null  object 
 9   categoriesIds              37840 non-null  object 
 10  link                       37840 non-null  object 
 11  Vendido por                0 non-null      object 
 12  Descripción del producto   37840 non-null  object 
 13  Presentación               355 non-null    object 


In [25]:
df['product_uri_json'] = df['productId'].apply(lambda x: f'https://www.wong.pe/api/catalog_system/pub/products/search?fq=productId:{x}')

In [26]:
df['product_uri_json'].sample(n=5)

25383    https://www.wong.pe/api/catalog_system/pub/pro...
5964     https://www.wong.pe/api/catalog_system/pub/pro...
7552     https://www.wong.pe/api/catalog_system/pub/pro...
18286    https://www.wong.pe/api/catalog_system/pub/pro...
30004    https://www.wong.pe/api/catalog_system/pub/pro...
Name: product_uri_json, dtype: object

In [27]:
file_name_productos_csv = f'productos_{fecha_hora_para_archivos}.csv'
df.to_csv(file_name_productos_csv, index=False)

In [28]:
df.iloc[26955]

productId                                                              1047747
productName                    Parrilla Carbon Beef Maker Plegable Rectangular
brand                                                               Beef Maker
brandId                                                                   1111
brandImageUrl                                                             None
categoryId                                                             1000692
productTitle                   Parrilla Carbon Beef Maker Plegable Rectangular
metaTagDescription           ¡Encuentra aquí la mejor variedad en productos...
categories                   [/Deportes y Outdoors/Terrazas y Parrillas/Par...
categoriesIds                [/1000069/1000689/1000692/, /1000069/1000689/,...
link                         https://www.wong.pe/parrilla-carbon-beef-maker...
Vendido por                                                               None
Descripción del producto                            